# To refactor
Not finshed yet

In [1]:
!nvidia-smi

Wed Jul  3 22:43:20 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.85                 Driver Version: 555.85         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   50C    P8             14W /   86W |     658MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [2]:
import torch
print(torch.cuda.is_available())

True


In [3]:
import mlflow
import os
import pandas as pd
from mlflow.tracking import MlflowClient
from dotenv import load_dotenv

# loading the .env file
load_dotenv() 

# load the env variables
os.environ['MLFLOW_TRACKING_USERNAME'] = os.getenv('MLFLOW_TRACKING_USERNAME')
os.environ['MLFLOW_TRACKING_PASSWORD'] = os.getenv('MLFLOW_TRACKING_PASSWORD')

# Set up authentication and tracking URI
dagshub_token = os.getenv('MLFLOW_TRACKING_PASSWORD')
username = "ssime-git"
repo_name = "mlops-introduction"
tracking_uri = f"https://dagshub.com/{username}/{repo_name}.mlflow"

# Set the MLflow tracking URI and authenticate
mlflow.set_tracking_uri(tracking_uri)
os.environ["MLFLOW_TRACKING_USERNAME"] = username
os.environ["MLFLOW_TRACKING_PASSWORD"] = dagshub_token

# Initialize MLflow client
client = MlflowClient()


In [4]:
# Model name and version
model_name = 'agnews-transformer-v2'
model_version = '1'  # or you can use "production", "staging", etc.

# Construct the model URI
model_uri = "mlflow-artifacts:/f1d16bb285c24918a9fae3a2612cdf38/fe91c1c1b9c848b5baacbd72b12efd56/artifacts/model"
#f"models:/{model_name}/{model_version}"

print(f"Model URI: {model_uri}")
print(f"MLFLOW_TRACKING_URI: {mlflow.get_tracking_uri()}")

# Fetch the model's environment dependencies
env_file_path = mlflow.pyfunc.get_model_dependencies(model_uri)
print(f"Environment file path: {env_file_path}")

# Load the model
model = mlflow.pyfunc.load_model(model_uri)


Model URI: mlflow-artifacts:/f1d16bb285c24918a9fae3a2612cdf38/fe91c1c1b9c848b5baacbd72b12efd56/artifacts/model
MLFLOW_TRACKING_URI: https://dagshub.com/ssime-git/mlops-introduction.mlflow


c:\Users\sebas\Documents\projet\mlops-introduction\mlflow_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024/07/03 22:44:07 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r C:\Users\sebas\AppData\Local\Temp\tmpu1za790z\model\requirements.txt'.


Environment file path: C:\Users\sebas\AppData\Local\Temp\tmpu1za790z\model\requirements.txt


2024/07/03 22:44:46 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - cloudpickle (current: 3.0.0, required: cloudpickle==2.2.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [11]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.pytorch
  run_id: fe91c1c1b9c848b5baacbd72b12efd56

In [ ]:
# not working at the moment
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Tokenizer needs to be loaded sepparetly for this
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def predict(texts, model, tokenizer):
    # Tokenize the texts
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    print(inputs)

    # Pass the inputs to the model
    with torch.no_grad():
        outputs = model.predict(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)

    # Convert predictions to text labels
    predictions = predictions.cpu().numpy()
    predictions = [model.config.id2label[prediction] for prediction in predictions]

    # Print predictions
    return predictions

# Sample text to predict
texts = [
    "The local high school soccer team triumphed in the state championship, securing victory with a last-second winning goal.",
    "DataCore is set to acquire startup InnovateAI for $2 billion, aiming to enhance its position in the artificial intelligence market.",
]

print(predict(texts, model, tokenizer))

In [6]:
import mlflow.pyfunc
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

class TransformersWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        self.model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
        self.tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    def predict(self, context, model_input):
        texts = model_input['text'].tolist()
        inputs = self.tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=-1)
        predictions = predictions.cpu().numpy()
        return predictions

# Assuming you have already loaded your model and tokenizer as shown above
import mlflow

# Save the model
with mlflow.start_run() as run:
    mlflow.pyfunc.save_model(
        path="transformers_model",
        python_model=TransformersWrapper(),
        artifacts={
            "model_path": "models/"
        },
    )

import pandas as pd


# Example input data
texts = [
    "The local high school soccer team triumphed in the state championship, securing victory with a last-second winning goal.",
    "DataCore is set to acquire startup InnovateAI for $2 billion, aiming to enhance its position in the artificial intelligence market.",
]
input_data = pd.DataFrame({"text": texts})

# Predict using the loaded model
predictions = model.predict(input_data)
print(predictions)


2024/07/03 22:45:46 WARNING mlflow.utils.requirements_utils: Found torch version (2.3.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.3.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


ValueError: could not convert string to float: 'The local high school soccer team triumphed in the state championship, securing victory with a last-second winning goal.'

In [10]:
import mlflow.pytorch
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

# Define the model URI
model_uri = "mlflow-artifacts:/f1d16bb285c24918a9fae3a2612cdf38/fe91c1c1b9c848b5baacbd72b12efd56/artifacts/model"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_uri)
model = AutoModelForSequenceClassification.from_pretrained(model_uri)

# Set the model to evaluation mode
model.eval()

# Define a function for inference
def predict(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    
    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the predicted label
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()
    label_name = model.config.id2label[predicted_label]
    return label_name

# Example usage
if __name__ == "__main__":
    text = "Sample input text for prediction"
    prediction = predict(text)
    print(f"Predicted label: {prediction}")


OSError: Incorrect path_or_model_id: 'mlflow-artifacts:/f1d16bb285c24918a9fae3a2612cdf38/fe91c1c1b9c848b5baacbd72b12efd56/artifacts/model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the model and tokenizer
custom_model = AutoModelForSequenceClassification.from_pretrained("models/agnews_transformer/model")
tokenizer = AutoTokenizer.from_pretrained("models/agnews_transformer/model")

def predict(texts, model, tokenizer):
    # Tokenize the texts
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(model.device)

    # Pass the inputs to the model
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)

    # Convert predictions to text labels
    predictions = predictions.cpu().numpy()
    predictions = [model.config.id2label[prediction] for prediction in predictions]

    # Print predictions
    return predictions

# Sample text to predict
texts = [
    "The local high school soccer team triumphed in the state championship, securing victory with a last-second winning goal.",
    "DataCore is set to acquire startup InnovateAI for $2 billion, aiming to enhance its position in the artificial intelligence market.",
]

